In [88]:
# Load the data, filter out columns which aren't useful,
# and convert categorial columns to a onehot representation

import pandas as pd
import numpy as np
import re

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

# Simplify some fields that are needlessly verbose
def simplify_field(f):
    f = f.replace("patient.cytogenetic_abnormalities.cytogenetic_abnormality",
        "patient.cytogenetic_abnormality")
    f = f.replace("patient.fish_test_component_results.fish_test_component_result",
        "patient.fish_test_component_result")
    f = f.replace("molecular_analysis_abnormality_testing_results.molecular_analysis_abnormality_testing_result_values",
        "molecular_analysis_abnormality_testing_result_values")
    f = f.replace("immunophenotype_cytochemistry_testing_results.immunophenotype_cytochemistry_testing_result_values",
        "immunophenotype_cytochemistry_testing_result")
    f = f.replace("race_list.race", "race")
    f = f.replace("patient.", "")
    return f

def clean_complex_bool_cols(df, base_name, sub_name, count, output=False):
    # Find unique values
    uniq_vals = set()
    for i in range(count):
        col = base_name + ("-%d" % (i+1) if i > 0 else "") + sub_name
        uniq_vals = uniq_vals.union(df[col].unique())
    if output:
        print(uniq_vals)

    # Turn index rows "", -2, -3 into True/False columns for each value
    new_df = pd.DataFrame()
    for val in uniq_vals:
        if pd.isnull(val):
            continue
        
        bool_vals = df[base_name + sub_name] == val
        for i in range(1, count):
            col = base_name + ("-%d" % (i+1) if i > 0 else "") + sub_name
            bool_vals = bool_vals | (df[col] == val)
        new_col = base_name + "_" + val
        new_df[new_col] = bool_vals
        if output:
            print(new_col, new_df[new_col])

    return new_df

def clean_complex_bool_cols_with_numeric(df, base_name, sub_name1, sub_name2, count, output=False):
    # Find unique values
    uniq_vals = set()
    for i in range(count):
        col = base_name + ("-%d" % (i+1) if i > 0 else "") + sub_name1
        uniq_vals = uniq_vals.union(df[col].unique())
    if output:
        print(uniq_vals)

    # Turn index rows "", -2, -3 into True/False columns for each value
    new_df = pd.DataFrame()
    for val in uniq_vals:
        if pd.isnull(val):
            continue
        
        mask = df[base_name + sub_name1] == val
        numeric_vals = pd.Series([np.nan] * df.shape[0], index=df.index)
        numeric_vals[mask] = df[base_name + sub_name2][mask]
        for i in range(1, count):
            mask = df[base_name + sub_name1] == val
            numeric_vals[mask] = df[base_name + sub_name2][mask]
        new_col = sub_name2[1:] + "_" + val

        if not pd.isnull(numeric_vals).all():
            new_df[new_col] = numeric_vals
            
        if output:
            print(new_col, new_df[new_col])

#     print(new_df.info())
    return new_df


# Get a onehot dataframe for a categorical column
def get_onehot_df(df, col):
    df = pd.get_dummies(df[col], drop_first=True)
    df.columns = [col + "_" + c for c in df.columns]    
    return df

# Create the pandas dataframe from a file
def load_dataframe(fn):
    df = pd.read_csv(open(fn), sep="\t").transpose()
    df.columns = df.iloc[0]
    df = df.drop(df.index[0]) # drop the row with the columns names

    df.columns = [simplify_field(c) for c in df.columns]
    df = df.apply(pd.to_numeric, errors='ignore') # convert object cols to numeric cols when possible
#     print(df.describe())
    
    # Remove some empty fields
    for field in ["molecular_analysis_abnormality_testing_results",
         "fish_test_component_results"]:
        del df[field]
    for field in df.columns:
        if field.startswith("admin.") or \
            "immunophenotype_cytochemistry_percent_positive" in field or \
            "molecular_analysis_abnormality_testing_percentage_value" in field:
            del df[field]

    # Clean complex columns with indices (e.g. "-2") into onehot dataframes
    ca_df = clean_complex_bool_cols(df, "cytogenetic_abnormality", "", 4)
    ictr_df = clean_complex_bool_cols(df, "immunophenotype_cytochemistry_testing_result", 
        ".immunophenotype_cytochemistry_testing_result", 21)
    ictr_df = ictr_df.drop(columns=[c for c in ictr_df.columns if "not tested" in c])
    maatr_df = clean_complex_bool_cols(df, "molecular_analysis_abnormality_testing_result_values", 
        ".molecular_analysis_abnormality_testing_result", 8)
    ftcr_df = clean_complex_bool_cols(df, "fish_test_component_result", ".fish_test_component", 9)
    ftcpv_df = clean_complex_bool_cols_with_numeric(df, "fish_test_component_result", 
        ".fish_test_component", ".fish_test_component_percentage_value", 9)
    complex_dfs = [ca_df, ictr_df, maatr_df, ftcr_df, ftcpv_df]
        
    ignore_col_prefixes = ["bcr_patient_barcode", "immunophenotype_cytochemistry_testing_result",
                           "fish_test_component_result", "molecular_analysis_abnormality_testing_result_values",
                           "cytogenetic_abnormality", "bcr_patient_uuid",
                           "days_to_death", "days_to_last_followup", # these are kind of cheating
                           "patient_id"]
    
    for c in df.columns:
        if re.split("[\.\-]", c)[0] in ignore_col_prefixes:
            del df[c]
    
    df = pd.concat([df] + complex_dfs, axis=1)
#     print(df.describe())
    
    final_df = pd.DataFrame()
    for c in df.columns:        
        # skip cols with no unique values
        if len(df[c].unique()) == 1:
            continue
        
        # convert categorical cols to onehot
        if df[c].dtype == object:
            new_df = get_onehot_df(df, c)
            final_df = pd.concat([final_df, new_df], axis=1)
        # skip cols with too many nans
        elif pd.isnull(df[c]).sum() > len(df[c]) * 0.8:
            continue
        # use numeric cols as is
        elif df[c].dtype in (np.float64, np.int64, np.bool):
            final_df = pd.concat([final_df, df[[c]]], axis=1)
        else:
            print("Col " + c + " is a weird type: " + str(df[c].dtype))
    
    return final_df

df = load_dataframe("LAML.merged_only_clinical_clin_format.txt")
print(list(df.columns))
print(df.shape)

['acute_myeloid_leukemia_calgb_cytogenetics_risk_category_intermediate/normal', 'acute_myeloid_leukemia_calgb_cytogenetics_risk_category_poor', 'age_at_initial_pathologic_diagnosis', 'atra_exposure_yes', 'cumulative_agent_total_dose', 'cytogenetic_abnormality_other_21', 'cytogenetic_abnormality_other_21, 9(9;22)', 'cytogenetic_abnormality_other_no', 'cytogenetic_abnormality_other_t(15;17)', 'cytogenetic_abnormality_other_t(4;11)', 'cytogenetic_abnormality_other_t(9;22)', 'cytogenetic_abnormality_other_yes', 'cytogenetic_analysis_performed_ind_yes', 'days_to_birth', 'ethnicity_not hispanic or latino', 'fish_evaluation_performed_ind_yes', 'fluorescence_in_situ_hybrid_cytogenetics_metaphase_nucleus_result_count', 'fluorescence_in_situ_hybridization_abnormal_result_indicator_yes', 'gender_male', 'germline_testing_performed_yes', 'history_of_neoadjuvant_treatment_yes', 'hydroxyurea_administration_prior_registration_clinical_study_indicator_yes', 'hydroxyurea_agent_administered_day_count', '

In [89]:
# Create our matrix of input features and vector of labels

y = df['vital_status_dead']
X = df[:]
del X['vital_status_dead']
print(df.shape, X.shape)

(200, 134) (200, 133)


In [90]:
# Impute missing values using the average and scale values to the same range

from sklearn.preprocessing import Imputer, StandardScaler
# Fit only to the training data
imp = Imputer()
Xi = imp.fit_transform(X)
print(Xi.shape)

scaler = StandardScaler()
Xis = scaler.fit_transform(Xi)
print(Xis.shape)
Xis = pd.DataFrame(Xis, columns=X.columns)

(200, 133)
(200, 133)


In [91]:
# Perform the classification with a few classifiers and use
# 10-fold cross validation with AUROC as the scoring metric.
# For ensemble methods, print out the top features.

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import RidgeClassifier, SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier

# Calculate metrics for the given classifiers fit to cross-validated subsets of an input feature matrix
def calc_clf_scores(X, y, clfs, cv, metric_names):
    print("Using %d features" % len(X.columns))

    mean_scores = pd.DataFrame(columns=[n for n in clfs], index=metric_names)
    important_features = None
    for clf_name in clfs:
        print(clf_name)
        for metric_name in metric_names:
            try:
                scores = cross_validate(clfs[clf_name], X, y, cv=cv, scoring=[metric_name])
                mean_score, std_score = np.mean(scores["test_"+metric_name]), np.std(scores["test_"+metric_name])
                mean_scores.ix[metric_name, clf_name] = mean_score
                print("%s: %f +-%f" % (metric_name, mean_score, std_score * 2))
            except AttributeError:
                pass

        clfs[clf_name].fit(X, y)
        print("Confusion matrix:\n", confusion_matrix(y, clfs[clf_name].predict(X)))

        if clf_name == "RandomForest":
            feat_importances = pd.Series(clfs[clf_name].feature_importances_, index=X.columns)
            feat_importances = feat_importances.sort_values(ascending=False)
            important_features = feat_importances.index
            print(feat_importances[:20])
        print()
    print()
    
    return mean_scores, important_features

seed = 11
clfs = {"NeuralNet": MLPClassifier(max_iter=1000, hidden_layer_sizes=(250,250,250,250), random_state=seed),
        "RandomForest": RandomForestClassifier(n_estimators=250, random_state=seed),
        "AdaBoost": AdaBoostClassifier(n_estimators=250, random_state=seed),
        "Bagging": BaggingClassifier(n_estimators=250, random_state=seed),
        "SVC": SVC(probability=True, random_state=seed),
        "Ridge": RidgeClassifier(random_state=seed),
        "SGDL1": SGDClassifier(penalty="l1", max_iter=1000, random_state=seed),
        "SGDL2": SGDClassifier(penalty="l2", max_iter=1000, random_state=seed),
        "LogisticL1a": LogisticRegression(penalty="l1", random_state=seed),
        "LogisticL2a": LogisticRegression(penalty="l2", random_state=seed),
        "LogisticL2b": LogisticRegression(penalty="l2", C=0.001, random_state=seed),
       }

voting_clfs = [(name, clfs[name]) for name in ("NeuralNet", "RandomForest", "SVC")]
voting_weights = [1, 1, 1]
clfs.update({
    "VotingSoft": VotingClassifier(voting_clfs, voting="soft", weights=voting_weights),
    "VotingHard": VotingClassifier(voting_clfs, voting="hard", weights=voting_weights)
})

skf = StratifiedKFold(n_splits=10, random_state=seed)
metric_names = ["f1", "roc_auc"]

scores_allfeat, important_features = calc_clf_scores(Xis, y, clfs, skf, metric_names)
print("Important features: ", " ".join(sorted(important_features)[:20]), "\n")
scores_50feat, dummy = calc_clf_scores(Xis[important_features[:50]], y, clfs, skf, metric_names)
scores_25feat, dummy = calc_clf_scores(Xis[important_features[:25]], y, clfs, skf, metric_names)
scores_10feat, dummy = calc_clf_scores(Xis[important_features[:10]], y, clfs, skf, metric_names)


Using 133 features
NeuralNet
f1: 0.725371 +-0.236030
roc_auc: 0.619362 +-0.270073
Confusion matrix:
 [[ 67   0]
 [  0 133]]

RandomForest
f1: 0.789049 +-0.109999
roc_auc: 0.674516 +-0.227973
Confusion matrix:
 [[ 67   0]
 [  0 133]]
age_at_initial_pathologic_diagnosis                              0.07
days_to_birth                                                    0.07
year_of_initial_pathologic_diagnosis                             0.05
platelet_result_count                                            0.04
lab_procedure_bone_marrow_neutrophil_result_percent_value        0.03
lab_procedure_blast_cell_outcome_percentage_value                0.03
lab_procedure_bone_marrow_blast_cell_outcome_percent_value       0.03
lab_procedure_leukocyte_result_unspecified_value                 0.03
lab_procedure_bone_marrow_lymphocyte_outcome_percent_value       0.03
lab_procedure_monocyte_result_percent_value                      0.02
lab_procedure_hemoglobin_result_specified_value                  0.

f1: 0.749013 +-0.184441
roc_auc: 0.666392 +-0.256523
Confusion matrix:
 [[ 67   0]
 [  0 133]]

SVC
f1: 0.829326 +-0.148675
roc_auc: 0.722711 +-0.319341
Confusion matrix:
 [[ 42  25]
 [  3 130]]

Ridge
f1: 0.792536 +-0.170024
roc_auc: 0.701230 +-0.326647
Confusion matrix:
 [[ 42  25]
 [ 13 120]]

SGDL1
f1: 0.777960 +-0.263695
roc_auc: 0.711015 +-0.326909
Confusion matrix:
 [[ 43  24]
 [ 18 115]]

SGDL2
f1: 0.762974 +-0.240272
roc_auc: 0.704369 +-0.342040
Confusion matrix:
 [[ 41  26]
 [ 15 118]]

LogisticL1a
f1: 0.807855 +-0.179029
roc_auc: 0.721088 +-0.309961
Confusion matrix:
 [[ 42  25]
 [ 15 118]]

LogisticL2a
f1: 0.811878 +-0.197204
roc_auc: 0.714704 +-0.306459
Confusion matrix:
 [[ 42  25]
 [ 15 118]]

LogisticL2b
f1: 0.766280 +-0.193608
roc_auc: 0.757195 +-0.256258
Confusion matrix:
 [[ 40  27]
 [ 24 109]]

VotingSoft
f1: 0.776119 +-0.169646
roc_auc: 0.734851 +-0.279752
Confusion matrix:
 [[ 67   0]
 [  0 133]]

VotingHard
f1: 0.814628 +-0.150724
Confusion matrix:
 [[ 67   0]
 [

In [92]:
# Compare the different metric's results using all the classifiers and different numbers of features

mean_scores = pd.Panel(items=["all", "50", "25", "10"], major_axis=metric_names, minor_axis=[n for n in clfs])
mean_scores["all"] = scores_allfeat
mean_scores["50"] = scores_50feat
mean_scores["25"] = scores_25feat
mean_scores["10"] = scores_10feat

pd.set_option("display.precision", 2)
print("roc_auc\n", mean_scores.ix[:,"roc_auc",:])
print("f1\n", mean_scores.ix[:,"f1",:])

roc_auc
                all    50    25    10
NeuralNet     0.62  0.73  0.72  0.71
RandomForest  0.67  0.71  0.71   0.7
AdaBoost      0.63   0.7  0.65  0.64
Bagging       0.67  0.68  0.67  0.67
SVC           0.64   0.7  0.72  0.72
Ridge         0.55  0.67   0.7   0.7
SGDL1         0.61  0.67  0.71   0.7
SGDL2          0.6  0.66   0.7  0.67
LogisticL1a   0.61  0.68  0.72   0.7
LogisticL2a   0.57  0.67  0.71   0.7
LogisticL2b   0.62  0.71  0.76  0.71
VotingSoft    0.69  0.74  0.73  0.73
VotingHard     NaN   NaN   NaN   NaN
f1
                all    50    25    10
NeuralNet     0.73  0.76  0.77  0.74
RandomForest  0.79  0.78  0.79  0.79
AdaBoost      0.72  0.75  0.75   0.7
Bagging       0.72  0.75  0.75  0.74
SVC           0.81   0.8  0.83  0.81
Ridge         0.64  0.73  0.79  0.78
SGDL1         0.66  0.72  0.78  0.77
SGDL2         0.72  0.71  0.76  0.76
LogisticL1a    0.7  0.74  0.81  0.76
LogisticL2a   0.66  0.74  0.81  0.77
LogisticL2b   0.64  0.71  0.77  0.74
VotingSoft    0.73  0.76 

In [48]:
# Answers to questions:
# 1. Classifiers I tried: NeuralNet, RandomForest, AdaBoost, SVC,
#    StochasticGradientDescent(L1 and L2), Logistic(L1 and L2), Ridge, and Voting
#    (soft and hard; combo of others). VotingSoft, SVC, and RandomForest
#    provided the best average AUROCs (0.75, 0.74, and 0.73) and F1 scores (0.8, 0.82, and 0.79)
#    when using the 25 top features but the AUROCs at least were still pretty variable across
#    the different cross validation sets (95% confidence in +-0.08). Note that there are a 
#    LOT of tests being run here so I wouldn't stake the business on this conclusion, but 
#    rather it's something that could be tested again on an independent data set. Also, this
#    is for only one random number seed...
# 2. I did a fair bit of preprocessing of the data before it was useful. I cleaned up the 
#    indexed columns (e.g. "cytogenetic_abnormality-2") into onehot columns for each possible
#    value. I also replaced other categorical data with onehot columns. Numeric columns were
#    left intact if they didn't have too man NaNs or the values weren't all the same. A few
#    other columns were removed e.g. ("patient_id") and finally "days_to_death" was removed
#    because this seemed like cheating.
# 3. I used scikit-learn methods to perform 5-fold cross validation stratified on the
#    vital_status label.
# 4. Some of the top features as returned by the ensemble methods (in alphabetical order):
#   age_at_initial_pathologic_diagnosis days_to_birth
#   lab_procedure_abnormal_lymphocyte_result_percent_value
#   lab_procedure_blast_cell_outcome_percentage_value
#   lab_procedure_bone_marrow_blast_cell_outcome_percent_value
#   lab_procedure_bone_marrow_lymphocyte_outcome_percent_value
#   lab_procedure_bone_marrow_metamyelocyte_result_value
#   lab_procedure_bone_marrow_neutrophil_result_percent_value
#   lab_procedure_bone_marrow_promonocyte_count_result_percent_value
#   lab_procedure_leukocyte_result_unspecified_value
#   lab_procedure_monocyte_result_percent_value
#   platelet_result_count year_of_initial_pathologic_diagnosis